# Clasificador de árbol de decisión
Este notebook tiene como función principal, entrenar los modelos de entreamiento con el algorítmo árbol de decisiones, evaluar el rendimiento de cada uno y almacenarlos en archivos con formato ".joblib".

Autor: Maider Murugarren Ilundain

In [7]:
# Importamos todas las librerías necesarias
import os
import pandas as pd
import joblib
import time
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix

In [8]:
# Función para entrenar los modelos con los conjuntos de datos
def entrenar_DecisionTreeClassifier(df):
    """
    Entrena diversos modelos de clasificación con el algorítmo árbol de decisiones,
    usando los conjuntos de características de los audios.

    Parámetros:
    - Dataframe: El conjunto de características de cada tipo de audio.

    Retorna:
    - Modelo entrenado.
    """
    
    # Dividir los datos en características (X) y etiquetas (y)
    # Además de borrar las caracteristicas sin relevancia para este entrenamiento
    X = df.drop(['Estadio', 'ID', 'archivo'], axis=1)
    y = df['Estadio']

    # Rellenar los valores vacios con la media del resto de valores
    X_filled = X.fillna(X.mean())

    # Crear dos nuevas columnas desde la columna género para que sean integros
    X_encoded = pd.get_dummies(X_filled, columns=['Genero'])

    # Crear una instancia del modelo arbol de decisiones
    tree_classifier = DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 1, min_samples_split = 2, ccp_alpha = 0)

    # Hacemos la validación cruzada para conseguir las predicciones
    y_pred = cross_val_predict(tree_classifier, X_encoded, y, cv=5)
    
    # Entrenamiento del modelo
    tree_classifier.fit(X_encoded, y)
    
    # Devolver modelo y los conjuntos de prueba
    return tree_classifier, X_encoded, y, y_pred

# Función para evaluar el rendimiento de los modelos
def evaluar_modelo(model, X, y, y_pred, nombre_modelo):
    
    """
    Obtiene diversas métricas de evaluación de los modelos entrenados, almacenandolos en un archivo con formato ".csv".

    Parámetros:
    - model: Modelo de árbol de decisiones entrenado.
    - X: conjunto de características del modelo.
    - y: característica con las etiquetas
    - y_pred: validación cruzada del modelo.
    - nombre_modelo: nombre que queramos poner al modelo entrenado.
    """
    
    # Creamos un diccionario para acumular los valores de las métricas
    metricas_eval_dict = {}
    
    y_scores = cross_val_predict(model, X, y, cv=5, method='predict_proba')[:, 1]

    # Calcular diversas métricas para evaluar el modelo
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    
    # Calcular los valores de la matriz de confusion
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    
    # Calcular los valores de la curva ROC
    fpr, tpr, thresholds = roc_curve(y, y_scores)
    
    # Almacenarlos todos en el diccionario
    metricas_eval_dict = {"Exactitud": accuracy,
                    "Precisión":precision,
                    "Sensibilidad": recall,
                    "Puntaje de F1": f1,
                    "TN": tn,
                    "FP": fp,
                    "FN": fn,
                    "TP": tp,
                    "FPR":fpr[1],
                    "TPR":tpr[1]}
    
    carpeta_csv = "Clasificador_arbol_decision"
    
    # Crear un archivo csv donde almacenar los valores de las métricas para cada tipo de modelo
    with open(carpeta_csv + "_metricas.csv", 'a', newline='') as archivo_csv:
        writer = csv.writer(archivo_csv, delimiter=';')
        
        archivo_vacio = archivo_csv.tell() == 0

        if archivo_vacio:
            # Escribir las métricas como encabezados de columnas
            writer.writerow(['modelo'] + list(metricas_eval_dict.keys()))

        # Escribir los valores de las métricas por fila
        writer.writerow([nombre_modelo] + list(metricas_eval_dict.values()))
        print("Metricas correctamente guardadas.")

# Función para guardar los modelos entrenados en formato ".joblib".       
def guardar_modelo(modelo, nombre_archivo):
    """
    Recupera los modelos entrenados en la función anterior y los almacena en la carpeta indicada con el nombre indicado. 
    Imprimiendo el mensaje del correcto almacenamiento y el nombre con el que se ha hecho.

    Parámetros:
    - model: Modelo de árbol de decisiones entrenado.
    - nombre_archivo: nombre que queramos poner al archivo almacenado.
    """
    joblib.dump(modelo, nombre_archivo)
    print(f"El modelo ha sido guardado correctamente en el archivo {nombre_archivo}.")


In [4]:
# Función para llamar a todas las funciones anteriores conjuntamente
def entrenar_y_evaluar_modelo(ruta_csv, nombre_modelo):
     """
    Esta función carga el conjunto de datos como dataframe y llama conjuntamente a todas las funciones anteriores.
    Parámetros:
    - ruta_csv: carpeta donde se encuentran los datos a utilizar.
    - nombre_modelo: nombre que queramos poner al modelo entrenado.
    """
    # Cargar los datos del CSV
    df = pd.read_csv(ruta_csv, delimiter=';')

    # Entrenar el modelo 
    modelo, X, y, y_pred = entrenar_DecisionTreeClassifier(df)

    # Evaluar el modelo
    evaluar_modelo(modelo, X, y, y_pred, nombre_modelo)

    # Guardar el modelo
    guardar_modelo(modelo, nombre_modelo + '.joblib')



In [13]:
# Entrenamiento de con todos los cvs con caracteristicas Spicy, para todos los tipos de audios

# La carpeta donde se encuentran los datos a utilizar
carpeta = "union_carac_Spicy/"

# Recorremos cada conjunto de datos
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

# Obtenemos el nombre del modelo de manera automática y llamamos al resto de funciones con ella
for archivo_csv in archivos_csv:
    ruta_csv = os.path.join(carpeta, archivo_csv)
    tipo = archivo_csv[-5]
    nombre_modelo = 'modelo_entrenado_DecisionTreeClassifier_tipo' + tipo + '_spicy'
    entrenar_y_evaluar_modelo(ruta_csv, nombre_modelo)


1
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo1_spicy.joblib.
2
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo2_spicy.joblib.
3
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo3_spicy.joblib.
4
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo4_spicy.joblib.
5
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo5_spicy.joblib.
6


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\106

Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo6_spicy.joblib.


In [15]:
# Entrenamiento de con todos los cvs con caracteristicas Pythorch, para todos los tipos de audios

# La carpeta donde se encuentran los datos a utilizar
carpeta = "unio_carac_Pythorch/"

# Recorremos cada conjunto de datos
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

# Obtenemos el nombre del modelo de manera automática y llamamos al resto de funciones con ella
for archivo_csv in archivos_csv:
    ruta_csv = os.path.join(carpeta, archivo_csv)
    tipo = archivo_csv[-5]
    nombre_modelo = 'modelo_entrenado_DecisionTreeClassifier_tipo' + tipo + '_Pythorch'
    entrenar_y_evaluar_modelo(ruta_csv, nombre_modelo)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\106

Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo1_Pythorch.joblib.
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo2_Pythorch.joblib.
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo3_Pythorch.joblib.
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo4_Pythorch.joblib.
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo5_Pythorch.joblib.
Metricas correctamente guardadas.
El modelo ha sido guardado correctamente en el archivo modelo_entrenado_DecisionTreeClassifier_tipo6_Pythorch.joblib.


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13784\1065049119.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_filled = X.fillna(X.mean())
